# import

In [1]:

import torch

from torchtext.vocab import Vocab
from torchtext._torchtext import (
    Vocab as VocabPybind,
)

from scgpt.trainer import eval_testdata as scgpt_test
from scgpt.preprocess import Preprocessor
from scgpt.model import TransformerModel
from scgpt.utils import load_pretrained
import sys
sys.path.append("./")
from scgpt_helper import *

import lamindb as ln

%load_ext autoreload
%autoreload 2

<frozen importlib._bootstrap>:914: ImportWarning: _SixMetaPathImporter.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _SixMetaPathImporter.find_spec() not found; falling back to find_module()
/home/ml4ig1/miniconda3/envs/training-gpt/lib/python3.10/site-packages/wandb/sdk/launch/builder/build.py:11: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/home/ml4ig1/miniconda3/envs/training-gpt/lib/python3.10/site-packages/pkg_resources/__init__.py:2871: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/home/ml4ig1/miniconda3/envs/training-gpt/lib/python3.10/site-packages/pkg_res

💡 lamindb instance: jkobject/scprint


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## create experiment

In [3]:
#experiment = scGExperiment()
batch_keys = [
    "self_reported_ethnicity_ontology_term_id",
    "assay_ontology_term_id",
]
special_tokens = ["<pad>", "<unk>", "<mask>"]
filter_gene_by_counts = 3
data_is_raw = True
save_path = "../save/"


In [4]:
vocab = "../save/scGPT_human/vocab.json"
model_path = "../save/scGPT_human/best_model.pt"

## prepare dataset

In [5]:
dataset_name = "retina"

In [6]:
adata = ln.File.filter(uid=ln.File.search("retina").index[0]).one().load()
# adata = ln.File.filter().first().load()
#ln.Dataset.using("laminlabs/cellxgene-census").one()
adata

<frozen importlib._bootstrap>:914: ImportWarning: _SixMetaPathImporter.find_spec() not found; falling back to find_module()


AnnData object with n_obs × n_vars = 19694 × 37127
    obs: 'n_genes', 'n_counts', 'percent_mito', 'donor_id', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'sex_ontology_term_id', 'tissue_ontology_term_id', 'author_cell_type', 'suspension_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage'
    var: 'chromosome', 'featureend', 'featurestart', 'n_cells', 'percent_cells', 'robust', 'highly_variable_features', 'mean', 'var', 'hvf_loess', 'hvf_rank', 'gene_symbols', 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype'
    uns: 'cell_type_ontology_term_id_colors', 'default_embedding', 'schema_version', 'title'
    obsm: 'X_diffmap', 'X_diffmap_pca', 'X_fitsne', 'X_fle', 'X_pca', 'X_phi', 'X_umap'

## setup

In [7]:
save_dir = setup(dataset_name, save_path, config)

if type(vocab) is str:
    vocab = GeneVocab.from_file(vocab)
vocab.set_default_index(vocab[config['pad_token']])

dataset = load_dataset(adata, vocab)
dataset.obs["batch_id"] = dataset.obs[batch_keys].apply("_".join, axis=1)

scGPT - INFO - save to ../save/_retina-Dec08-15-13
scGPT - INFO - match 25037/37127 genes in vocabulary of size 60697.


/home/ml4ig1/Documents code/scGPT/mytests/scgpt_helper.py:141: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  dataset.var["gene_ids"] = vocab(dataset.var["gene_symbols"].tolist())


In [8]:
# experiment.init_datamodule(dataset=, vocab=)

In [10]:
preprocessor = Preprocessor(
    use_key="X",  # the key in adata.layers to use as raw data
    filter_gene_by_counts=filter_gene_by_counts,  # step 1
    filter_cell_by_counts=False,  # step 2
    normalize_total=1e4,  # 3. whether to normalize the raw data and to what sum
    result_normed_key="X_normed",  # the key in adata.layers to store the normalized data
    log1p=data_is_raw,  # 4. whether to log1p the normalized data
    result_log1p_key="X_log1p",
    subset_hvg=config['n_hvg'],  # 5. whether to subset the raw data to highly variable genes
    hvg_flavor="seurat_v3" if data_is_raw else "cell_ranger",
    binning=config['n_bins'],  # 6. whether to bin the raw data and to what number of bins
    result_binned_key="X_binned",  # the key in adata.layers to store the binned data
)

preprocessor(dataset)

scGPT - INFO - Filtering genes by counts ...
scGPT - INFO - Normalizing total counts ...
scGPT - INFO - Log1p transforming ...
scGPT - INFO - Subsetting highly variable genes ...
scGPT - WARNING - No batch_key is provided, will use all cells for HVG selection.
scGPT - INFO - Binning data ...


9

In [18]:
model = TransformerModel(
    len(vocab),  # n_tokens
    # TODO:
    config['embsize'],
    config['nhead'],
    config['d_hid'],
    config['nlayers'],
    vocab=vocab,
    n_cls=len(dataset.obs.cell_type.value_counts()),
    dropout=config['dropout'],
    pad_token=config['pad_token'],
    pad_value=config['pad_value'],
    do_mvc=config['MVC'],
    do_dab=True,
    use_batch_labels=True,
    num_batch_labels=len(set(dataset.obs["batch_id"])),
    #domain_spec_batchnorm=config['dsbn'],
    n_input_bins=config['n_bins'],
    #ecs_threshold=config['ecs_thres'],
    #explicit_zero_prob=config['explicit_zero_prob'],
    use_fast_transformer=config['fast_transformer'],
    #pre_norm=config['pre_norm'],
)
if model_path is not None:
    load_pretrained(model, torch.load(model_path), verbose=False)
    # model_config['file'] = model_dir / "args.json"
    # model_file = model_dir / "best_model.pt"
model.to(device)

hey


/home/ml4ig1/miniconda3/envs/training-gpt/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer was not TransformerEncoderLayer
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


TransformerModel(
  (encoder): GeneEncoder(
    (embedding): Embedding(60697, 512, padding_idx=60694)
    (enc_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (value_encoder): ContinuousValueEncoder(
    (dropout): Dropout(p=0.2, inplace=False)
    (linear1): Linear(in_features=1, out_features=512, bias=True)
    (activation): ReLU()
    (linear2): Linear(in_features=512, out_features=512, bias=True)
    (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (batch_encoder): BatchLabelEncoder(
    (embedding): Embedding(1, 512)
    (enc_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-11): 12 x FlashTransformerEncoderLayer(
        (self_attn): FlashMHA(
          (Wqkv): Linear(in_features=512, out_features=1536, bias=True)
          (inner_attn): FlashAttention()
          (out_proj): Linear(in_features=512, out_features=512, bias=True)
        )
        (li

## fine tune and save

In [26]:
model = fine_tune(
    model, 
    config,
    dataset,
    vocab,
    config['epochs'],
    config['batch_size'],
    save_path,
    device,)

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f9b427717e0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f9b43649a20, raw_cell="fine_tune(
    model, 
    config,
    dataset,
  .." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bperso/home/ml4ig1/Documents%20code/scGPT/mytests/cleanup_run.ipynb#X30sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

/home/ml4ig1/miniconda3/envs/training-gpt/lib/python3.10/site-packages/wandb/sdk/lib/ipython.py:77: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display  # type: ignore


<frozen importlib._bootstrap>:914: ImportWarning: _SixMetaPathImporter.find_spec() not found; falling back to find_module()


/home/ml4ig1/miniconda3/envs/training-gpt/lib/python3.10/site-packages/wandb/sdk/lib/ipython.py:89: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


/home/ml4ig1/miniconda3/envs/training-gpt/lib/python3.10/site-packages/wandb/sdk/lib/ipython.py:77: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display  # type: ignore


scGPT - INFO - train set number of samples: 15755, 
	 feature length: 2001
scGPT - INFO - valid set number of samples: 3939, 
	 feature length: 2001
hey
hey
random masking at epoch   1, ratio of masked values in train:  0.3998
scGPT - INFO - valid/loss: 23.255262877499884epoch 1
scGPT - INFO - -----------------------------------------------------------------------------------------
scGPT - INFO - | end of epoch   1 | time: 354.72s | valid loss/mse 23.2553
scGPT - INFO - -----------------------------------------------------------------------------------------
scGPT - INFO - Best model with score 23.2553
scGPT - INFO - train set number of samples: 15755, 
	 feature length: 2001
scGPT - INFO - valid set number of samples: 3939, 
	 feature length: 2001
hey
hey
random masking at epoch   2, ratio of masked values in train:  0.3998
scGPT - INFO - valid/loss: 23.05727909433869epoch 2
scGPT - INFO - -----------------------------------------------------------------------------------------
scGPT 

/home/ml4ig1/miniconda3/envs/training-gpt/lib/python3.10/site-packages/wandb/sdk/lib/ipython.py:77: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display  # type: ignore


train/cls,▁▃▄▁▄▁█▁▁▅▂▂▂▅▁▄▁▁▁▁▂▁▁▂▅▁▃▁▄▁▂▁▁▁▂▂▁▂▂▃
train/gep,▂▅█▂▄▇▁▂▅▂▆▄▃▂▅▃▂▂▂▁▃▄▃▂▆▄▂▂▆▅▂▁▂▁▂▅▃▂▄▄
train/cls,0.33092
train/gep,28.76307


970



In [27]:
load_pretrained(model, torch.load(save_path+"/model_e6.pt"), verbose=False)
model.to(device)

TransformerModel(
  (encoder): GeneEncoder(
    (embedding): Embedding(60697, 512, padding_idx=60694)
    (enc_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (value_encoder): ContinuousValueEncoder(
    (dropout): Dropout(p=0.2, inplace=False)
    (linear1): Linear(in_features=1, out_features=512, bias=True)
    (activation): ReLU()
    (linear2): Linear(in_features=512, out_features=512, bias=True)
    (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (batch_encoder): BatchLabelEncoder(
    (embedding): Embedding(1, 512)
    (enc_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-11): 12 x FlashTransformerEncoderLayer(
        (self_attn): FlashMHA(
          (Wqkv): Linear(in_features=512, out_features=1536, bias=True)
          (inner_attn): FlashAttention()
          (out_proj): Linear(in_features=512, out_features=512, bias=True)
        )
        (li

## look at what we have

In [ ]:
scgpt_test(
    best_model, adata_test, gene_ids, vocab, config
)